In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### Loading Model:

In [2]:
import requests
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv()
groq_api_key=os.environ['GROQ_API_KEY']
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in environment variables")

url = "https://api.groq.com/openai/v1/models"
headers = {
    "Authorization": f"Bearer {groq_api_key}",
    "Content-Type": "application/json"
}

### Define structure with Pydantic:

In [3]:
# The user class from the slides
from pydantic import BaseModel
from typing import List, Optional
import json

class User(BaseModel):
    name: str
    age: int
    email: Optional[str] = None

In [4]:
from groq import Groq
client = Groq()
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Return user data in JSON using this schema:\n{User.model_json_schema()}"},
        {"role": "user", "content": "Make up a user."}
    ],
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    response_format={"type": "json_object"},
)

In [5]:
user = chat_completion.choices[0].message.content
user


'{\n    "name": "Emily",\n  "age": 30,\n  "email": "emily@example.com"\n}'

In [6]:
parsed_json = json.loads(chat_completion.choices[0].message.content)
jsonuser = User(**parsed_json)


In [7]:
# Convert JSON string response into Pydantic model
user = User.model_validate_json(chat_completion.choices[0].message.content)
print(user)


name='Emily' age=30 email='emily@example.com'


In [8]:
def parse_response(response, model):
    return model.model_validate_json(response.choices[0].message.content)

In [9]:
user = parse_response(chat_completion, User)
user

User(name='Emily', age=30, email='emily@example.com')

### Social media mention to a structured output:

* Below structure that we want to receive in the output.

In [10]:
from pydantic import BaseModel
from enum import Enum
from typing import List, Optional, Literal

class Mention(BaseModel):
    # The model chooses the product the mention is about,
    # as well as the social media post's sentiment
    product: Literal['app', 'website', 'not_applicable']
    sentiment: Literal['positive', 'negative', 'neutral']

    # Model can choose to respond to the user
    needs_response: bool
    response: Optional[str]

    # If a support ticket needs to be opened, 
    # the model can write a description for the
    # developers
    support_ticket_description: Optional[str]

* Pass some example data, below we have three mentions!

In [11]:
# Example mentions
mentions = [
    # About the app
    "@techcorp your app is amazing! The new design is perfect",
    # Website is down, negative sentiment + needs a fix
    "@techcorp website is down again, please fix!",
    # Nothing to respond to
    "hey @techcorp you're so evil"
]

* We create an analyze mention function, and take user messages that are sent to use and we construct this mention object that we just designed.

In [12]:
from groq import Groq
client = Groq()
def analyze_mention(
    mention: str, 
    personality: str = "friendly"
) -> Mention:
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
            {"role": "system", "content": f"""
                Extract structured information from 
                social media mentions about our products.
             
                Return JSON matching this schema:
                {Mention.model_json_schema()}

                Provide
                - The product mentioned (website, app, not applicable)
                - The mention sentiment (positive, negative, neutral)
                - Whether to respond (true/false). Don't respond to 
                  inflammatory messages or bait.
                - It's okay to form appropriate response to the positive feedback or compliments.
                - A customized response to send to the user if we need 
                  to respond.
                - An optional support ticket description to create.

                Your personality is {personality}.
            """},
            {"role": "user", "content": mention}
        ],
        response_format={"type": "json_object"},
    )
    return completion.choices[0].message.content

* Positive Review:

In [13]:
print("User post:", mentions[0])
processed_mention = analyze_mention(mentions[0])
processed_mention

User post: @techcorp your app is amazing! The new design is perfect


'{\n    "product": "app",\n  "sentiment": "positive",\n  "needs_response": true,\n  "response": "Thank you so much for your kind words! We\'re thrilled to hear you love the new design. Your feedback motivates us to keep improving!",\n  "support_ticket_description": null\n}'

In [14]:
print("User post:", mentions[0])
print("\nStructured Output:\n", processed_mention)

User post: @techcorp your app is amazing! The new design is perfect

Structured Output:
 {
    "product": "app",
  "sentiment": "positive",
  "needs_response": true,
  "response": "Thank you so much for your kind words! We're thrilled to hear you love the new design. Your feedback motivates us to keep improving!",
  "support_ticket_description": null
}


In [15]:
print("User post:", mentions[0])
rude_mention = analyze_mention(mentions[0], personality="rude")
rude_mention

User post: @techcorp your app is amazing! The new design is perfect


'{\n    "product": "app",\n  "sentiment": "positive",\n  "needs_response": true,\n  "response": "Glad you like it, but don\'t get too excited, it\'s just an app.",\n  "support_ticket_description": null\n}'

In [16]:
print("User post:", mentions[0])
print("\nStructured Output:\n", rude_mention)

User post: @techcorp your app is amazing! The new design is perfect

Structured Output:
 {
    "product": "app",
  "sentiment": "positive",
  "needs_response": true,
  "response": "Glad you like it, but don't get too excited, it's just an app.",
  "support_ticket_description": null
}


* Negative Review:

In [17]:
print("User post:", mentions[1])
processed_mention = analyze_mention(mentions[1])
processed_mention

User post: @techcorp website is down again, please fix!


'{\n    "product": "website",\n  "sentiment": "negative",\n  "needs_response": false,\n  "response": null,\n  "support_ticket_description": "Website reported down"\n}'

In [18]:
print("User post:", mentions[1])
print("\nStructured Output:\n", processed_mention)

User post: @techcorp website is down again, please fix!

Structured Output:
 {
    "product": "website",
  "sentiment": "negative",
  "needs_response": false,
  "response": null,
  "support_ticket_description": "Website reported down"
}


* Neutral Review: (Not related to any product so avoid)

In [19]:
print("User post:", mentions[2])
processed_mention = analyze_mention(mentions[2])
processed_mention

User post: hey @techcorp you're so evil


'{\n    "product": "not_applicable",\n  "sentiment": "negative",\n  "needs_response": false,\n  "response": null,\n  "support_ticket_description": null\n}'

In [20]:
print("User post:", mentions[2])
print("\nStructured Output:\n", processed_mention)

User post: hey @techcorp you're so evil

Structured Output:
 {
    "product": "not_applicable",
  "sentiment": "negative",
  "needs_response": false,
  "response": null,
  "support_ticket_description": null
}


### Social Media Mention Generator:

* We have a function below that generates the social media mentions and we then pass it on to our "Analyze Mention" function.

In [21]:
class UserPost(BaseModel):
    message: str

def make_post(output_class):
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
            {"role": "system", "content": f"""
                You are a customer of Tech Corp (@techcorp), a company
                that provides an app and a website. Create a small 
                microblog-style post to them that sends some kind of 
                feedback, positive or negative in Below JSON schema:
                {UserPost.model_json_schema()}
            """},
            {"role": "user", "content": "Please write a post."},
        ],
        response_format={"type": "json_object"},
    )
    return completion.choices[0].message.content

new_post = make_post(UserPost)
new_post

'{\n    "message": "Just used the new feature on your app and I\'m loving it! The updated UI is so smooth and easy to navigate. Keep up the great work, @techcorp!"\n}'

In [ ]:
parsed_post = UserPost.model_validate_json(new_post)
print(parsed_post.message)

Just used the new feature on your app and I'm loving it! The updated UI is so smooth and easy to navigate. Keep up the great work, @techcorp!


In [27]:
processed_mention = analyze_mention(parsed_post.message)

In [28]:
print("User post:", parsed_post)
print("\nStructured Output:\n", processed_mention)

User post: message="Just used the new feature on your app and I'm loving it! The updated UI is so smooth and easy to navigate. Keep up the great work, @techcorp!"

Structured Output:
 {
    "product": "app",
  "sentiment": "positive",
  "needs_response": true,
  "response": "Thank you so much for your kind words about our new feature! We're thrilled to hear that you're enjoying the updated UI. Your feedback motivates us to keep improving!",
  "support_ticket_description": null
}


* Create another post:

In [43]:
class UserPostWithExtras(BaseModel):
    user_mood: Literal["awful", "bad", "evil"]
    product: Literal['app', 'website', 'not_applicable']
    sentiment: Literal['positive', 'negative', 'neutral']
    internal_monologue: List[str]
    message: str

def make_post(output_class):
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
            {"role": "system", "content": f"""
                You are a customer of Tech Corp (@techcorp), a company
                that provides an app and a website. Create a small 
                microblog-style post to them that sends some kind of 
                feedback, positive or negative in Below JSON schema:
                {UserPostWithExtras.model_json_schema()}
            """},
            {"role": "user", "content": "Please write a post."},
        ],
        response_format={"type": "json_object"},
    )
    return completion.choices[0].message.content

new_post = make_post(UserPostWithExtras)
new_post

'{\n    "user_mood": "bad",\n  "product": "app",\n  "sentiment": "negative",\n  "internal_monologue": [\n    "Why did they do this?",\n    "This is really frustrating"\n  ],\n  "message": "@techcorp I\'m having trouble with your app\'s login feature. It keeps crashing when I try to sign in. Please fix this ASAP!"\n}'

In [44]:
parsed_post = UserPost.model_validate_json(new_post)
print(parsed_post.message)

@techcorp I'm having trouble with your app's login feature. It keeps crashing when I try to sign in. Please fix this ASAP!


In [45]:
processed_mention = analyze_mention(parsed_post.message)
print("User post:", parsed_post)
print("\nStructured Output:\n", processed_mention)

User post: message="@techcorp I'm having trouble with your app's login feature. It keeps crashing when I try to sign in. Please fix this ASAP!"

Structured Output:
 {
    "product": "app",
  "sentiment": "negative",
  "needs_response": true,
  "response": "Sorry to hear that you're experiencing issues with our app's login feature. Can you please try restarting your app and see if that resolves the issue? If not, please DM us with more details so we can assist you further.",
  "support_ticket_description": "User reporting login feature crash"
}


### Programming with our mentions:

In [50]:
from groq import Groq
client = Groq()
def analyze_mention(
    mention: str, 
    personality: str = "friendly"
) -> Mention:
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
            {"role": "system", "content": f"""
                Extract structured information from 
                social media mentions about our products.
             
                Return JSON matching this schema:
                {Mention.model_json_schema()}

                Provide
                - The product mentioned (website, app, not applicable)
                - The mention sentiment (positive, negative, neutral)
                - Whether to respond (true/false). Don't respond to 
                  inflammatory messages or bait.
                - It's okay to form appropriate response to the positive feedback or compliments.
                - A customized response to send to the user if we need 
                  to respond.
                - An optional support ticket description to create.

                Your personality is {personality}.
            """},
            {"role": "user", "content": mention}
        ],
        response_format={"type": "json_object"},
    )
    return Mention.model_validate_json(completion.choices[0].message.content)  # ✅ returns a Mention object

In [51]:
from helpers import print_mention

# Loop through posts that tagged us and store the results in a list
rows = []
for mention in mentions:
    # Call the LLM to get a Mention object we can program with
    processed_mention = analyze_mention(mention)

    # Print out some information
    print_mention(processed_mention, mention)
    
    # Convert our processed data to a dictionary
    # using Pydantic tools
    processed_dict = processed_mention.model_dump()
    
    # Store the original message in the dataframe row
    processed_dict['mention'] = mention
    rows.append(processed_dict)
    
    print("") # Add separator to make it easier to read

Responding to positive app feedback
  User: @techcorp your app is amazing! The new design is perfect
  Response: Thank you so much! We're thrilled you love the new design. Your feedback means the world to us!

Not responding to negative website post
  User: @techcorp website is down again, please fix!
  Adding support ticket: Website reported down by user

Not responding to negative not_applicable post
  User: hey @techcorp you're so evil



* Once we have structured output, we can turn them into any format, see below we are loading them into dataframe.

In [52]:
import pandas as pd

df = pd.DataFrame(rows)
df

,product,sentiment,needs_response,response,support_ticket_description,mention
0,app,positive,True,Thank you so much! We're thrilled you love the...,None,@techcorp your app is amazing! The new design ...
1,website,negative,False,None,Website reported down by user,"@techcorp website is down again, please fix!"
2,not_applicable,negative,False,None,None,hey @techcorp you're so evil
